In [1]:
import requests
import json
from PIL import Image
from io import BytesIO
import base64
import numpy as np 
import cupy as cp
import sys
from cuvs.neighbors import hnsw,cagra
import os
import time
sys.path.append('/hpc2hdd/home/ysi538/retrieval')
from MDI_RAG_Image2Image_Research.src.utils.encoder import WSI_Image_UNI_Encoder
""" # 请求数据
# url = '  /hpc/retrieval/query/image2image_retrieval'
url = 'http://localhost:9876/test'
query_img_path = " metaservice/api/region/openslide/241183-21.tiff/6400/25344/256/256/1"
request_data = {
    'query_img_path': query_img_path,
    'top_k': 20
}

# 发送 POST 请求
response = requests.post(url, json=request_data)

# 检查响应状态码
if response.status_code == 200:
    response_data = response.json()
    print("Response JSON:", response_data)
else:
    print("Request failed with status code:", response.status_code) """

/hpc2hdd/home/ysi538/miniconda3/envs/yhre-new/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


' # 请求数据\n# url = \'  /hpc/retrieval/query/image2image_retrieval\'\nurl = \'http://localhost:9876/test\'\nquery_img_path = " metaservice/api/region/openslide/241183-21.tiff/6400/25344/256/256/1"\nrequest_data = {\n    \'query_img_path\': query_img_path,\n    \'top_k\': 20\n}\n\n# 发送 POST 请求\nresponse = requests.post(url, json=request_data)\n\n# 检查响应状态码\nif response.status_code == 200:\n    response_data = response.json()\n    print("Response JSON:", response_data)\nelse:\n    print("Request failed with status code:", response.status_code) '

In [7]:
def get_combined_regions(result_infos):

    search_info_list = []
    
    for info in result_infos:
        search_info = {}
        level = info.split("_")[-1].split(".")[0]
        w = info.split("_")[-3]
        h = info.split("_")[-2]
        x = info.split("_")[-5]
        y = info.split("_")[-4]
        id = info.split("_")[0]
        name = "_".join(info.split("_")[1:-5])
        search_info = {"id": id, "name": name, "x": x, "y": y, "w": w, "h": h, "level": level}
        search_info_list.append(search_info)

    def dfs(node, component, visited):
        # 将当前节点标记为已访问
        visited[node] = True
        # 将当前节点添加到当前组件
        component.append(search_info_list[node])
        
        # 遍历其他所有节点
        for neighbor in range(len(search_info_list)):
            if not visited[neighbor] and judge_if_connected(search_info_list[node], search_info_list[neighbor]):
                # 如果邻居节点未被访问过且与当前节点相连，则递归地进行DFS
                dfs(neighbor, component, visited)

    # 初始化变量
    visited = [False] * len(search_info_list)
    components = []

    # 对每个节点执行DFS，如果该节点尚未被访问
    for i in range(len(search_info_list)):
        if not visited[i]:
            # 创建一个新的组件列表
            current_component = []
            # 执行DFS并填充当前组件
            dfs(i, current_component, visited)
            # 将当前组件添加到结果列表
            components.append(current_component)

    # 剔除掉只有一个region的component
    components = [component for component in components if len(component) > 1]

    return components

def judge_if_connected(info1, info2):
    # 判断两个region是否相邻,先判断name是否相同，再判断level是否相同，再判断左上角坐标的xy加上或者减去wh是否与另一个region的xy相同
    if info1["name"] != info2["name"]:
        return False
    if info1["level"] != info2["level"]:
        return False
    if int(info1["x"]) + int(info1["w"]) == int(info2["x"]) and int(info1["y"]) == int(info2["y"]):
        return True
    if int(info1["x"]) - int(info1["w"]) == int(info2["x"]) and int(info1["y"]) == int(info2["y"]):
        return True
    if int(info1["y"]) + int(info1["h"]) == int(info2["y"]) and int(info1["x"]) == int(info2["x"]):
        return True
    if int(info1["y"]) - int(info1["h"]) == int(info2["y"]) and int(info1["x"]) == int(info2["x"]):
        return True
    if int(info1["x"]) + int(info1["w"]) == int(info2["x"]) and int(info1["y"]) + int(info1["h"]) == int(info2["y"]):
        return True
    if int(info1["x"]) - int(info1["w"]) == int(info2["x"]) and int(info1["y"]) - int(info1["h"]) == int(info2["y"]):
        return True
    if int(info1["x"]) + int(info1["w"]) == int(info2["x"]) and int(info1["y"]) - int(info1["h"]) == int(info2["y"]):
        return True
    if int(info1["x"]) - int(info1["w"]) == int(info2["x"]) and int(info1["y"]) + int(info1["h"]) == int(info2["y"]):
        return True
    return False

def request_image(query_img_path):
    if "http" in query_img_path:
        response = requests.get(query_img_path, verify=False)
        query_image = Image.open(BytesIO(response.content)).convert("RGB")
    else:
        query_image = Image.open(query_img_path).convert("RGB")
    return query_image

def split_img(query_image, other_info):
    imgs = []
    # 以无重叠的方式切割图片为多个正方形
    w, h = query_image.size
    if w == h:
        return [query_image]
    if w > h:
        new_w = h
        new_h = h
    else:
        new_w = w
        new_h = w
    for i in range(0, w, new_w):
        for j in range(0, h, new_h):
            box = (i, j, i + new_w, j + new_h)
            img = query_image.crop(box)
            imgs.append(img)
    return imgs

def get_img_other_info(query_path):
    pass
    return None




def load_index_file(index_file):
        index = hnsw.load(index_file, 1024, np.float32, "sqeuclidean")
        return index

def load_info_file(info_file):
    with open (info_file, 'r') as f:
        infos = json.load(f)
    return infos



def search(index, encoder, infos_list, query_path, top_k=20):
    total_neighbors = []
    total_distances = []
    query_image = request_image(query_path)
    # 根据图片比例尺寸，切割图片
    img_other_info = get_img_other_info(query_path)
    split_imgs = split_img(query_image, img_other_info)
    # 编码后的向量存储到一个二维 np.array 中
    split_embeddings = cp.array([cp.array(encoder.encode_image(after_split_img)).astype('float32') for after_split_img in split_imgs])
    
    # 确保每个 query_embedding 是二维数组
    if split_embeddings.ndim == 1:
        split_embeddings = split_embeddings.reshape(1, -1)
    
    # print("split_embeddings:", split_embeddings)
    # return split_embeddings
    result_infos = []
    # search_params = hnsw.SearchParams()
    
    time_cost = 0
    begin_time = time.time()
    # distances, neighbors = hnsw.search(search_params, index, split_embeddings, top_k)
    distances, neighbors = cagra.search(cagra.SearchParams(),index, split_embeddings, top_k)
    end_time = time.time()
    time_cost += end_time - begin_time
    
    neighbors = cp.asarray(neighbors).flatten().tolist()
    distances = cp.asarray(distances).flatten().tolist()
    for neighbor in neighbors:
        result_infos.append(infos_list[neighbor])
    total_neighbors.extend(neighbors)
    total_distances.extend(distances)

    total_distances, total_neighbors, result_infos = zip(*sorted(zip(total_distances, total_neighbors, result_infos)))
    return total_distances[:], total_neighbors[:], result_infos[:]






In [3]:
query_img_path = " metaservice/api/region/openslide/241183-21.tiff/6400/25344/512/256/1"
image_encoder = WSI_Image_UNI_Encoder()

ssd_dir = "/hpc2ssd/JH_DATA/spooler/ysi538/"

hnsw_index = load_index_file(ssd_dir + "cupy_index_batch_0.bin")
infos_list = load_info_file(ssd_dir + "cupy_infos_batch_0.json")



cuda


In [4]:
embeddings = cp.load(ssd_dir + "cupy_embeddings_batch_0.npy")
# 转换为numpy
embeddings = cp.asnumpy(embeddings).astype(np.float32)
embeddings = cp.array(embeddings, dtype=cp.float32)
build_params = cagra.IndexParams(metric="sqeuclidean",build_algo = 'nn_descent')
cuda_index = cagra.build(build_params, embeddings)

[I] [15:53:42.278367] optimizing graph
[I] [15:54:07.391248] Graph optimized, creating index


In [8]:


total_distances, total_neighbors, result_infos = search(cuda_index, image_encoder, infos_list, query_img_path, 20)



combined_regions = get_combined_regions(result_infos)

/hpc2hdd/home/ysi538/miniconda3/envs/yhre-new/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mdi.hkust-gz.edu.cn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [43]:
# total_distances, total_neighbors, result_infos 
combined_regions

[[{'id': '6947',
   'name': '2113040-79_无淋巴结转移.tiff',
   'x': '22272',
   'y': '3584',
   'w': '256',
   'h': '256',
   'level': '1'},
  {'id': '6947',
   'name': '2113040-79_无淋巴结转移.tiff',
   'x': '22272',
   'y': '3328',
   'w': '256',
   'h': '256',
   'level': '1'}],
 [{'id': '47062',
   'name': 'EGFR-肺癌-230214010TFXA-0.7-LBP.ibl.tiff',
   'x': '9984',
   'y': '16640',
   'w': '256',
   'h': '256',
   'level': '1'},
  {'id': '47062',
   'name': 'EGFR-肺癌-230214010TFXA-0.7-LBP.ibl.tiff',
   'x': '10240',
   'y': '16896',
   'w': '256',
   'h': '256',
   'level': '1'}],
 [{'id': '7505',
   'name': 'D23-00779-21_无淋巴结转移.tiff',
   'x': '11776',
   'y': '9472',
   'w': '256',
   'h': '256',
   'level': '1'},
  {'id': '7505',
   'name': 'D23-00779-21_无淋巴结转移.tiff',
   'x': '12032',
   'y': '9472',
   'w': '256',
   'h': '256',
   'level': '1'}]]

In [34]:

# 问题1:切分时粒度如何确定？
# 问题2:切分后检索结果是否考虑原始位置关系？



# 我们的方法流程大致如下：
# 1.获得一张病理图片
# 2.根据某种切分方式切分图片（可创新）
# 3.将切分后的图片分别进行编码，使用Uni
# 4.将每个编码进行向量检索，通过batch的方式进行检索（可创新，使用GPU进行加速）
# 5.将检索结果通过某种方式进行合并，得到region的检索结果（可创新，使用GPU进行加速，带相对位置信息的dfs+多数投票，可强调权重以展示通用性和框架性）



# 问题1:切分时粒度如何确定？
# 问题2:检索时如何加速检索？
# 问题3:如何将检索结果合并为region的检索结果？

# 对比指标：
# 1.检索速度和精度的曲线（time-recall)，和传统cpu方法进行对比，和对gpu检索方法进行对比
# 2.和单个patch的检索结果对比相似度，以及部分示例（考虑缩略图的特殊情况？）


        
